In [1]:
import pandas as pd
import time
from time import sleep
import os,sys
from threading import Thread
from google.colab import output,drive
import datetime
import matplotlib.pyplot as plt
import numpy as np
import joblib
import datetime

!pip install smartapi-python
!pip install pyotp
!pip install logzero
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib

output.clear()

In [2]:
from tensorflow.keras.models import Sequential, model_from_json
json_file = open('SBIN_model_1min_97.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("SBIN_model_1min_97.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp

api_key = 'Your_API_Key'
clientId = 'You_Client_Id'
pwd = 'Your_AngelOne_Password'
smartApi = SmartConnect(api_key)
smartApi._routes["api.market.data"]="/rest/secure/angelbroking/market/v1/quote"
token = "Your_TOTP_Token"
totp=pyotp.TOTP(token).now()
correlation_id = "hdhdjdjdjd"

# login api call

data = smartApi.generateSession(clientId, pwd, totp)
# print(data)
authToken = data['data']['jwtToken']
refreshToken = data['data']['refreshToken']

# fetch the feedtoken
feedToken = smartApi.getfeedToken()

# fetch User Profile
res = smartApi.getProfile(refreshToken)
smartApi.generateToken(refreshToken)
res=res['data']['exchanges']

In [ ]:
# class CandleData():
#     ans={}
#     OPEN=-1
#     prev_OPEN=-1
#     CLOSE=-1
#     prev_CLOSE=-1
#     HIGH=-1
#     prev_HIGH=-1
#     LOW=-1
#     prev_LOW=-1
#     trend=''
#     prev_trend=''

# candle=CandleData()
# interval=300
# ticks=40
# # Fetch Feed Data
# buy_price=0
# sell_price=0
# stop_loss=0
# transactions=[]

# param={
#   "mode": "OHLC",
#   "exchangeTokens": {
#     "NSE": [
#       "26000"
#     ]
#   }
# }

In [ ]:
# def buy_stock(val):
#     global buy_price,stop_loss
#     latest=smartApi._request('api.market.data',"POST",param)
#     buy_price=latest['data']['fetched'][0]['ltp']
#     stop_loss=val
#     print('Bought ','Stop_loss:',stop_loss,' ',time.time())

# def sell_stock():
#     global buy_price,sell_price,transactions,stop_loss
#     latest=smartApi._request('api.market.data',"POST",param)
#     sell_price=latest['data']['fetched'][0]['ltp']
#     transaction=[buy_price,sell_price,sell_price-buy_price]
#     transactions.append(transaction)
#     buy_price=0
#     sell_price=0
#     stop_loss=0
#     print('Sold',' ',time.time())
#     print('CP:',transaction[0],'SP:',transaction[1],'P:',transaction[2])

In [ ]:
'''def market_feed(candle):
    global interval,ticks,stop_loss
    candle.OPEN=candle.CLOSE
    candle.HIGH=candle.CLOSE
    candle.LOW=candle.CLOSE
    start=time.time()
    while(time.time()-start<interval):
        candle.ans=smartApi._request('api.market.data',"POST",param)
        price=candle.ans['data']['fetched'][0]['ltp']
        if candle.OPEN==-1:
            candle.OPEN=price
        candle.CLOSE=price
        candle.HIGH=max(candle.HIGH,price)
        candle.LOW=min(candle.LOW,price)
        #if at any point of time the price hits stop_loss,
        #i will simply sell the stock to stop my loss
        if stop_loss>=price:
            sell_stock()

def getCandleStick(candle):
    global ticks,buy_price,sell_price,stop_loss

    while True:
        market_feed(candle)
  ###### start operation #####

      #BULLISH (prev is red, current is green that crosses prev's high)
      #buying conditon: previous trend indicates change, and no stock
      # at hand at the moment
        if candle.CLOSE>candle.OPEN and candle.CLOSE>candle.prev_HIGH:
            if candle.prev_trend!='' and buy_price==0:
                buy_stock(candle.prev_LOW)
        s=identify(candle)
        candle.trend=''
        if s!='':
            candle.trend=Trend[s]
            #print(s,' ',Trend[s])
        if candle.prev_trend=='bearish' and buy_price!=0:
            if candle.CLOSE<candle.prev_LOW:
                sell_stock()
        if buy_price!=0 and candle.CLOSE>=buy_price+3*(buy_price-stop_loss):
            sell_stock()
        print(candle.OPEN,' ',candle.CLOSE,' ',candle.HIGH,' ',candle.LOW,' ',s,' ',candle.trend)

   ###### end operation #####
        candle.prev_CLOSE=candle.CLOSE
        candle.prev_OPEN=candle.OPEN
        candle.prev_HIGH=candle.HIGH
        candle.prev_LOW=candle.LOW
        candle.prev_trend=candle.trend
#Getting starting Candle to avoid low=-1
candle.ans=smartApi._request('api.market.data',"POST",param)
price=candle.ans['data']['fetched'][0]['ltp']
candle.OPEN=price
candle.CLOSE=price
candle.HIGH=price
candle.LOW=price
'''

Next would be to achieve the 7EMA and 21EMA of latest data.

In [ ]:
from datetime import datetime,timedelta
import talib
import numpy as np
start=time.time()

def history(historicParam):
    try:
        return smartApi.getCandleData(historicParam)
    except Exception as e:
        print("Historic Api failed: {}".format(e.message))

from datetime import datetime,timedelta,date
import time

t=datetime.now()
#df=datetime(t.year,t.month,t.day,9,15)
t=datetime(t.year,t.month,t.day,t.hour,t.minute)
t=t+timedelta(hours=5,minutes=30)
#print(t)
arr=[]

historicParam={    "exchange": "NSE",
    "symboltoken": "3045",
    "interval": "ONE_MINUTE",
    "fromdate": "",
    "todate": ""
    }
fromdate=datetime(t.year,t.month,t.day,9,15)
fromdate=fromdate-timedelta(days=5)
todate=t
#print(fromdate,' ',todate)
historicParam['fromdate']=fromdate.strftime("%Y-%m-%d %H:%M")
historicParam['todate']=todate.strftime("%Y-%m-%d %H:%M")

hist=history(historicParam)
#print(hist)
hist=hist['data']
print(len(hist))
columns=['timestamp','open','high','low','close','volume']
data=pd.DataFrame(hist,columns=columns)
data['timestamp']=pd.to_datetime(data['timestamp'],format="%Y-%m-%d %H:%M")
val={}
Columns=['timestamp','close']
for col in Columns:
    val[col]=np.array(data[col]).flatten()
ind=np.array(data.index).flatten()
df=pd.DataFrame(val,index=ind)
#sc=StandardScaler()
#df[Columns]=sc.fit_transform(df[Columns])
#for i in range(len(df)):
    #df['close'][i]=df['close'][i]*0.001
print(time.time()-start)

1187
1.0851867198944092


In [ ]:
_7EMA=talib.EMA(np.array(df['close']),7)
_21EMA=talib.EMA(np.array(df['close']),20)

In [ ]:

buy=0
sell=0
loss=0
k=1
last_7EMA=_7EMA[-1]
last_21EMA=_21EMA[-1]
last_timestamp=df['timestamp'][len(df)-1]
org=np.array(df['close'][-5:]).flatten()
times=[]
while True:
    # Fetching latest interval data
    t=datetime.now()

    fromtime=datetime(t.year,t.month,t.day,t.hour,t.minute)
    fromtime=fromtime+timedelta(hours=5,minutes=30)
    totime=datetime(t.year,t.month,t.day,t.hour,t.minute)
    totime=totime+timedelta(hours=5,minutes=31)
    historicParam['fromdate']=fromtime.strftime("%Y-%m-%d %H:%M")
    historicParam['todate']=totime.strftime("%Y-%m-%d %H:%M")
    present=history(historicParam)
    present=present['data']
    columns=['timestamp','open','high','low','close','volume']
    present=pd.DataFrame(present,columns=columns)
    present['timestamp']=pd.to_datetime(present['timestamp'],format="%Y-%m-%d %H:%M")

    if len(present)>0 and last_timestamp!=present['timestamp'][0]:
        l=last_7EMA
        r=last_21EMA
        last_timestamp=present['timestamp'][0]
        #print(last_timestamp)
        last_7EMA=0.25*present['close'][0]+0.75*last_7EMA
        last_21EMA=(2/21)*present['close'][0]+(1-2/21)*last_21EMA
        org=org[1:]
        org=np.append(org,present['close'][0])
        for k in range(5):
            org[k]=org[k]*0.001

        org=org.reshape((1,5,))

        pred_next=mod.predict(org)*1000+0.661

        EMA7=0.25*pred_next+0.75*last_7EMA
        EMA21=(2/21)*pred_next+(1-2/21)*last_21EMA
        print(present['timestamp'][0],' ',present['close'][0],' ',last_7EMA,' ',last_21EMA,' Pred:',pred_next)
        if (last_7EMA<last_21EMA and EMA7>EMA21) or (l<r and last_7EMA>last_21EMA):
            angle1=(EMA7-last_7EMA)-(EMA21-last_21EMA)
            angle2=(last_7EMA-l)-(last_21EMA-r)
            if angle1>0.13 or angle2>0.13:
                print('BOUGHT AT:',last_timestamp)
        org=np.array(org).flatten()
        for k in range(5):
            org[k]=org[k]*1000
#     continue
#     #break
#     for j in range(5):
#         org[j]=org[j]*0.001
#         #print(j)
#     org=org.reshape((1,5,))
#     #pred_next=mod.predict(org)
#     #print(df['close'][i+1],' ',pred_next*1000)
#     pred_7EMA=_7EMA[i+1]
#     pred_21EMA=_21EMA[i+1]
#     if _7EMA[i]<_21EMA[i] and pred_7EMA>pred_21EMA:
#         angle=(pred_7EMA-_7EMA[i])-(pred_21EMA-_21EMA[i])
#         #print(angle,' ',df['timestamp'][i])
#         if angle>0.13:
#             times.append([df['timestamp'][i],_7EMA[i],_21EMA[i],pred_7EMA,pred_21EMA])
# #output.clear()
# for i in times:
#     print(i)

1/1 [==============================] - 0s 24ms/step
2023-08-29 10:17:00+05:30   573.0   572.9681642657458   572.9521845126868  Pred: [[572.9973]]
1/1 [==============================] - 0s 46ms/step
2023-08-29 10:18:00+05:30   572.65   572.8886231993093   572.923405035288  Pred: [[572.91046]]
1/1 [==============================] - 0s 32ms/step
2023-08-29 10:19:00+05:30   572.65   572.828967399482   572.8973664604987  Pred: [[572.817]]
1/1 [==============================] - 0s 25ms/step
2023-08-29 10:20:00+05:30   572.55   572.7592255496115   572.8642839404513  Pred: [[572.71436]]
1/1 [==============================] - 0s 27ms/step
2023-08-29 10:21:00+05:30   572.6   572.7194191622086   572.8391140413606  Pred: [[572.647]]
1/1 [==============================] - 0s 26ms/step
2023-08-29 10:22:00+05:30   572.6   572.6895643716565   572.8163412755167  Pred: [[572.604]]
1/1 [==============================] - 0s 33ms/step
2023-08-29 10:23:00+05:30   572.6   572.6671732787423   572.795737344515

KeyboardInterrupt: ignored